In [36]:
import pandas as pd
pd.options.display.max_columns=200
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

In [37]:
data = pd.read_csv('data/btc_data', index_col=0)

In [38]:
data

,price,market_cap,progression daily,progression tomorrow,volumeto,target
2013-04-29,141.960000,1.575032e+09,0.049224,-0.046915,9.983975e+06,0
2013-04-30,135.300000,1.501657e+09,-0.046915,-0.135255,9.119201e+06,0
2013-05-01,117.000000,1.298952e+09,-0.135255,-0.115983,2.178794e+07,0
2013-05-02,103.430000,1.148668e+09,-0.115983,-0.120081,2.493210e+07,0
2013-05-03,91.010000,1.011066e+09,-0.120081,0.222393,2.249626e+07,1
...,...,...,...,...,...,...
2023-10-09,27948.103652,5.449897e+11,-0.001052,-0.012678,6.039641e+08,0
2023-10-10,27593.782534,5.382827e+11,-0.012678,-0.007304,5.111095e+08,0
2023-10-11,27392.247703,5.352004e+11,-0.007304,-0.020081,8.213956e+08,0
2023-10-12,26842.190439,5.236564e+11,-0.020081,-0.004212,4.385320e+08,0


In [34]:
import numpy as np
data['price_log'] = np.log(data['price'])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data['price_normalized'] = scaler.fit_transform(data[['price']])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data['price_standardized'] = scaler.fit_transform(data[['price']])

In [35]:
data

,price,market_cap,progression daily,progression tomorrow,volumeto,target,price_log,price_normalized,price_standardized
2013-04-29,141.960000,1.575032e+09,0.049224,-0.046915,9.983975e+06,0,4.955545,0.001098,-0.776361
2013-04-30,135.300000,1.501657e+09,-0.046915,-0.135255,9.119201e+06,0,4.907495,0.000999,-0.776788
2013-05-01,117.000000,1.298952e+09,-0.135255,-0.115983,2.178794e+07,0,4.762174,0.000728,-0.777963
2013-05-02,103.430000,1.148668e+09,-0.115983,-0.120081,2.493210e+07,0,4.638895,0.000527,-0.778834
2013-05-03,91.010000,1.011066e+09,-0.120081,0.222393,2.249626e+07,1,4.510969,0.000343,-0.779631
...,...,...,...,...,...,...,...,...,...
2023-10-09,27948.103652,5.449897e+11,-0.001052,-0.012678,6.039641e+08,0,10.238105,0.412741,1.008527
2023-10-10,27593.782534,5.382827e+11,-0.012678,-0.007304,5.111095e+08,0,10.225346,0.407495,0.985783
2023-10-11,27392.247703,5.352004e+11,-0.007304,-0.020081,8.213956e+08,0,10.218015,0.404512,0.972847
2023-10-12,26842.190439,5.236564e+11,-0.020081,-0.004212,4.385320e+08,0,10.197730,0.396369,0.937538


In [24]:
data['price'].describe()

count     3819.000000
mean     12236.613558
std      15580.688835
min         67.809000
25%        573.527400
50%       6481.323142
75%      19214.062986
max      67617.015545
Name: price, dtype: float64

In [40]:
# Créer une variable cible binaire : 1 si la progression demain est positive, 0 sinon
data['target'] = np.where(data['progression tomorrow'] > 0, 1, 0)

# Sélectionner les caractéristiques et exclure la dernière ligne
features = data.drop(columns=['progression tomorrow', 'target', 'price']).iloc[:-1, :]
target = data['target'].iloc[:-1]

In [39]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Définir la taille de la fenêtre initiale
window_size = 1000  # par exemple

# Initialiser les listes pour stocker les prédictions et les vraies valeurs
predictions = []
actuals = []

# Boucle à travers les données de la taille de la fenêtre jusqu'à la fin des données
for i in range(window_size, len(data) - 1):
    # Diviser les données en ensembles d'entraînement et de test
    X_train = features.iloc[i-window_size:i, :]
    y_train = target.iloc[i-window_size:i]
    X_test = features.iloc[i:i+1, :]
    y_test = target.iloc[i]
    
    # Entraîner un modèle
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    
    # Faire une prédiction
    prediction = model.predict(X_test)[0]
    
    # Stocker les prédictions et les vraies valeurs
    predictions.append(prediction)
    actuals.append(y_test)

# Évaluer le modèle
accuracy = accuracy_score(actuals, predictions)
print(f'Model Accuracy: {accuracy * 100:.2f}%')


KeyboardInterrupt: 

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialiser les listes pour stocker les prédictions, les probabilités de prédiction et les vraies valeurs
predictions = []
prediction_probs = []
actuals = []

# Boucle à travers les données de la taille de la fenêtre jusqu'à la fin des données
for i in range(window_size, len(data) - 1):
    # Diviser les données en ensembles d'entraînement et de test
    X_train = features.iloc[i-window_size:i, :]
    y_train = target.iloc[i-window_size:i]
    X_test = features.iloc[i:i+1, :]
    y_test = target.iloc[i]
    
    # Entraîner un modèle
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Faire une prédiction
    prediction = model.predict(X_test)[0]
    prediction_prob = model.predict_proba(X_test)[0][1]  # probabilité de la classe 1
    
    # Stocker les prédictions, les probabilités de prédiction et les vraies valeurs
    predictions.append(prediction)
    prediction_probs.append(prediction_prob)
    actuals.append(y_test)

# Évaluer le modèle
accuracy = accuracy_score(actuals, predictions)
print(f'Model Accuracy: {accuracy * 100:.2f}%')

Model Accuracy: 53.02%


In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

# Initialiser les listes pour stocker les prédictions, les probabilités de prédiction et les vraies valeurs
predictions = []
prediction_probs = []
actuals = []

# Supposons que window_size et data sont définis
window_size = 100  # exemple de taille de fenêtre

# Boucle à travers les données de la taille de la fenêtre jusqu'à la fin des données
for i in range(window_size, len(data) - 1):
    # Diviser les données en ensembles d'entraînement et de test
    X_train = features.iloc[i-window_size:i, :]
    y_train = target.iloc[i-window_size:i]
    X_test = features.iloc[i:i+1, :]
    y_test = target.iloc[i]
    
    # Entraîner un modèle
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Faire une prédiction
    prediction = model.predict(X_test)[0]
    prediction_prob = model.predict_proba(X_test)[0][1]  # probabilité de la classe 1
    
    # Stocker les prédictions, les probabilités de prédiction et les vraies valeurs
    predictions.append(prediction)
    prediction_probs.append(prediction_prob)
    actuals.append(y_test)

# Évaluer le modèle
accuracy = accuracy_score(actuals, predictions)
print(f'Model Accuracy: {accuracy * 100:.2f}%')

# Matrice de coût
cost_matrix = np.array([[0, 1],  # coût de classer 0 comme 0/1
                        [5, 0]])  # coût de classer 1 comme 0/1

# Matrice de confusion
conf_matrix = confusion_matrix(actuals, predictions)

# Coût total de mauvaise classification
total_cost = np.sum(conf_matrix * cost_matrix)
print(f'Total Cost of Misclassification: {total_cost}')

Model Accuracy: 52.66%
Total Cost of Misclassification: 5008
